# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
import dice_ml
from dice_ml import Dice
from dice_ml.utils import helpers # helper functions

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Preliminaries: Loading the data and ML model

In [3]:
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])


In [5]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
model = clf.fit(x_train, y_train)
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}
query_instances = x_train[0:3]
e1 = exp.generate_counterfactuals(query_instances, total_CFs=2, posthoc_sparsity_param=None)
e1.visualize_as_dataframe(show_only_changes=True)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,19,Other/Unknown,Some-college,Single,Other/Unknown,White,Female,30,0.0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39.0,Self-Employed,Masters,Married,Professional,-,Male,5.0,0.565
1,44.0,Self-Employed,Masters,Married,Professional,-,Male,98.0,0.721


Query instance (original outcome : 0)


...

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39.0,Self-Employed,Masters,-,Professional,-,-,5.0,0.565
1,44.0,Self-Employed,Masters,-,Professional,-,-,98.0,0.721


Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,62,Private,Some-college,Married,White-Collar,White,Male,40,0.525



Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,66.0,Government,Bachelors,Single,-,-,-,48.0,0.363
1,39.0,Government,Bachelors,Single,-,-,-,7.0,0.018


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.feature_importance(query_instances, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'workclass': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.0, 'gender': 1.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 0.0, 'occupation': 1.0, 'race': 0.0, 'gender': 0.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 0.0, 'race': 0.0, 'gender': 0.0, 'age': 1.0, 'hours_per_week': 1.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.feature_importance(query_instances, posthoc_sparsity_param=None)
print(imp.local_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
[{'workclass': 0.8, 'education': 1.0, 'marital_status': 0.7, 'occupation': 0.9, 'race': 0.2, 'gender': 0.8, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 0.5, 'occupation': 1.0, 'race': 0.2, 'gender': 0.2, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 0.9, 'education': 0.9, 'marital_status': 0.9, 'occupation': 0.6, 'race': 0.3, 'gender': 0.3, 'age': 1.0, 'hours_per_week': 1.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj=exp.feature_importance(query_instances[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
{'workclass': 0.9, 'education': 0.9666666666666667, 'marital_status': 0.7, 'occupation': 0.8333333333333334, 'race': 0.23333333333333334, 'gender': 0.43333333333333335, 'age': 1.0, 'hours_per_week': 1.0}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"model_type\": \"classifier\", \"desired_class\": 1.0, \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":19},\\\"workclass\\\":{\\\"0\\\":\\\"Other\\\\/Unknown\\\"},\\\"education\\\":{\\\"0\\\":\\\"Some-college\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Single\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Other\\\\/Unknown\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Female\\\"},\\\"hours_per_week\\\":{\\\"0\\\":30},\\\"income\\\":{\\\"0\\\":0.0}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":39.0,\\\"1\\\":44.0,\\\"2\\\":70.0,\\\"3\\\":74.0,\\\"4\\\":79.0,\\\"5\\\":60.0,\\\"6\\\":53.0,\\\"7\\\":53.0,\\\"8\\\":74.0,\\\"9\\\":48.0},\\\"workclass\\\":{\\\"0\\\":\\\"Self-Employed\\\",\\\"1\\\":\\\"Self-Employed\\\",\\\"2\\\":\\\"Self-Employed\\\",\\\"3\\\":\\\"Government\\\",\\\"4\\\":\\\"Self-Employed\\\",\\\"5\\\":\\\"Government\\\",\\\"6\\\":\\\"Government\\\",\\\"7\\\":\\\"Self-Employed\\\",\\\"8\\\":\\\"Ot

## Convert the json output to a counterfactual object

In [11]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe() for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

AttributeError: 'NoneType' object has no attribute 'outcome_name'